In [1]:
import numpy as np
import pandas as pd
import lxml.etree as etree
import sqlite3 as sql
from preprocess import DB
import matplotlib.pyplot as plt

In [2]:
db = DB(db_route = '../data/database.sqlite')

In [3]:
db.unravel()

NameError: name 'tqdm' is not defined

In [6]:
df1 = pd.merge(db.match_team, fouls0, how = 'left', left_on = 'match_api_id', right_on = 'match_id')
pd.merge(df1, cards0, how = 'left', left_on = 'match_api_id', right_on = 'match_id')

,match_api_id,date,home_team_api_id,home_team_name,away_team_api_id,away_team_name,foulcommit,card,corner,match_id_x,...,comment,ycards,event_incident_typefk_card,card_elapsed,card_type,card_subtype,card_player,card_team,card_type,card_id
0,489042,2008-08-17 00:00:00,10260,Manchester United,10261,Newcastle United,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,489042.0,...,y,1,73,78.0,y,serious_fouls,24157,10260,card,379481
1,489042,2008-08-17 00:00:00,10260,Manchester United,10261,Newcastle United,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,489042.0,...,y,1,73,82.0,y,serious_fouls,30362,10260,card,379503
2,489042,2008-08-17 00:00:00,10260,Manchester United,10261,Newcastle United,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,489042.0,...,y,1,70,90.0,y,None,30829,10260,card,379547
3,489042,2008-08-17 00:00:00,10260,Manchester United,10261,Newcastle United,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,489042.0,...,y,1,73,78.0,y,serious_fouls,24157,10260,card,379481
4,489042,2008-08-17 00:00:00,10260,Manchester United,10261,Newcastle United,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,489042.0,...,y,1,73,82.0,y,serious_fouls,30362,10260,card,379503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14289,1992218,2016-05-16 00:00:00,10199,FC Luzern,9931,FC Basel,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<corner />,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14290,1992188,2016-04-17 00:00:00,10190,FC St. Gallen,9931,FC Basel,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<corner />,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14291,1992139,2015-12-06 00:00:00,10191,FC Thun,9931,FC Basel,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<corner />,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14292,1992158,2016-02-28 00:00:00,10191,FC Thun,9931,FC Basel,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<corner />,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
cards0

,match_id,comment,ycards,event_incident_typefk_card,card_elapsed,card_type,card_subtype,card_player,card_team,card_type,card_id
0,489042,y,1,73,78,y,serious_fouls,24157,10260,card,379481
1,489042,y,1,73,82,y,serious_fouls,30362,10260,card,379503
2,489042,y,1,70,90,y,None,30829,10260,card,379547
